In [28]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

#Compute embedding for both lists
embeddings1 = model.encode(sents, convert_to_tensor=True)

a = np.matmul(embeddings1,embeddings1.T)
a = np.abs(a).numpy()
a = np.sum(a, axis = 1)
for i in np.argpartition(a, -10)[-10:]:
    print(sents[i])
#Compute cosine-similarities
# cosine_scores = util.cos_sim(embeddings1, embeddings2)

# #Output the pairs with their score
# for i in range(len(sentences1)):
#     print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

Deep learning is a hot topic these days.
Neural networks are multi-layer networks of neurons (the blue and magenta nodes in the chart below) that we use to classify things, make predictions, etc.
Below is the diagram of a simple neural network with five inputs, 5 outputs, and two hidden layers of neurons.
And by the end, hopefully you (and I) will have gained a deeper and more intuitive understanding of how neural networks do what they do.
In this post, we will explore the ins and outs of a simple neural network.
Our second hidden layer of neurons in magenta.
Neural networks are the workhorses of deep learning.
The arrows that connect the dots shows how all the neurons are interconnected and how data travels from the input layer all the way through to the output layer.
To even begin to answer it, we will need to learn the basics of neural networks.
We will also watch how the neural network learns from its mistake using a process known as backpropagation.


In [23]:
np.matmul(embeddings1[0], embeddings1[0])

tensor(1.)

In [3]:
import pandas as pd

In [4]:
df = pd.read_pickle('data.df')

In [15]:
from nltk import tokenize
p = "Good morning Dr. Adams. The patient is waiting for you in room number 3."

tokenize.sent_tokenize(p)

['Good morning Dr. Adams.', 'The patient is waiting for you in room number 3.']

In [24]:
# i = 
para = """Deep learning is a hot topic these days. But what is it that makes it special and sets it apart from other aspects of machine learning? That is a deep question (pardon the pun). To even begin to answer it, we will need to learn the basics of neural networks.

Neural networks are the workhorses of deep learning. And while they may look like black boxes, deep down (sorry, I will stop the terrible puns) they are trying to accomplish the same thing as any other model — to make good predictions.

In this post, we will explore the ins and outs of a simple neural network. And by the end, hopefully you (and I) will have gained a deeper and more intuitive understanding of how neural networks do what they do.
Let’s start with a really high level overview so we know what we are working with. Neural networks are multi-layer networks of neurons (the blue and magenta nodes in the chart below) that we use to classify things, make predictions, etc. Below is the diagram of a simple neural network with five inputs, 5 outputs, and two hidden layers of neurons.
Starting from the left, we have:

The input layer of our model in orange.
Our first hidden layer of neurons in blue.
Our second hidden layer of neurons in magenta.
The output layer (aka the prediction) of our model in green.
The arrows that connect the dots shows how all the neurons are interconnected and how data travels from the input layer all the way through to the output layer.

Later we will calculate step by step each output value. We will also watch how the neural network learns from its mistake using a process known as backpropagation.
"""
title = 'understanding neural networks'
print(title)

understanding neural networks


In [26]:
sents = tokenize.sent_tokenize(para)
sents

['Deep learning is a hot topic these days.',
 'But what is it that makes it special and sets it apart from other aspects of machine learning?',
 'That is a deep question (pardon the pun).',
 'To even begin to answer it, we will need to learn the basics of neural networks.',
 'Neural networks are the workhorses of deep learning.',
 'And while they may look like black boxes, deep down (sorry, I will stop the terrible puns) they are trying to accomplish the same thing as any other model — to make good predictions.',
 'In this post, we will explore the ins and outs of a simple neural network.',
 'And by the end, hopefully you (and I) will have gained a deeper and more intuitive understanding of how neural networks do what they do.',
 'Let’s start with a really high level overview so we know what we are working with.',
 'Neural networks are multi-layer networks of neurons (the blue and magenta nodes in the chart below) that we use to classify things, make predictions, etc.',
 'Below is the 

In [26]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

corpus_embeddings = embedder.encode(sents, convert_to_tensor=True)

# Query sentences:
queries = ['']


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(10, len(sents))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 10 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(sents[idx], "(Score: {:.4f})".format(score))





Query: 

Top 10 most similar sentences in corpus:
Let’s start with a really high level overview so we know what we are working with. (Score: 0.1488)
And while they may look like black boxes, deep down (sorry, I will stop the terrible puns) they are trying to accomplish the same thing as any other model — to make good predictions. (Score: 0.1299)
Our second hidden layer of neurons in magenta. (Score: 0.1161)
Starting from the left, we have:

The input layer of our model in orange. (Score: 0.1094)
Our first hidden layer of neurons in blue. (Score: 0.1005)
Neural networks are the workhorses of deep learning. (Score: 0.0761)
In this post, we will explore the ins and outs of a simple neural network. (Score: 0.0651)
We will also watch how the neural network learns from its mistake using a process known as backpropagation. (Score: 0.0607)
And by the end, hopefully you (and I) will have gained a deeper and more intuitive understanding of how neural networks do what they do. (Score: 0.0589)

# cdj

In [41]:
#### Pytextrank to find important phrases

import spacy
import pytextrank

# example text
text = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."

# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_lg")

# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")
doc = nlp(text)

# examine the top-ranked phrases in the document
for phrase in doc._.phrases:
    print(phrase.text)
    print(phrase.rank, phrase.count)
    print(phrase.chunks)

mixed types
0.18359439311764025 1
[mixed types]
systems
0.1784796193107821 3
[systems, systems, systems]
minimal generating sets
0.15037838042245094 1
[minimal generating sets]
nonstrict inequations
0.14740065982407313 1
[nonstrict inequations]
strict inequations
0.13946027725597837 1
[strict inequations]
linear Diophantine equations
0.1195023546245721 1
[linear Diophantine equations]
natural numbers
0.11450088293222845 1
[natural numbers]
solutions
0.10780718173686318 3
[solutions, solutions, solutions]
linear constraints
0.10529828014583348 1
[linear constraints]
all the considered types systems
0.1036960590708142 1
[all the considered types systems]
a minimal supporting set
0.08812713074893187 1
[a minimal supporting set]
a system
0.08243620500315359 1
[a system]
a minimal set
0.07944607954086784 1
[a minimal set]
algorithms
0.0763527926213032 1
[algorithms]
all types
0.07593126037016427 1
[all types]
Diophantine
0.07309361902551355 1
[Diophantine]
construction
0.0702090100898443 1


## tf-idf approach which helps to summarize text

In [10]:
text = """Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's struggle against Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic and subjugate all wizards and Muggles (non-magical people).

The series was originally published in English by Bloomsbury in the United Kingdom and Scholastic Press in the United States. All versions around the world are printed by Grafica Veneta in Italy.[1] A series of many genres, including fantasy, drama, coming of age, and the British school story (which includes elements of mystery, thriller, adventure, horror, and romance), the world of Harry Potter explores numerous themes and includes many cultural meanings and references.[2] According to Rowling, the main theme is death.[3] Other major themes in the series include prejudice, corruption, and madness.[4]

Since the release of the first novel, Harry Potter and the Philosopher's Stone, on 26 June 1997, the books have found immense popularity, positive reviews, and commercial success worldwide. They have attracted a wide adult audience as well as younger readers and are often considered cornerstones of modern young adult literature.[5] As of February 2018, the books have sold more than 500 million copies worldwide, making them the best-selling book series in history, and have been translated into more than eighty languages.[6] The last four books consecutively set records as the fastest-selling books in history, with the final instalment selling roughly 2.7 million copies in the United Kingdom and 8.3 million copies in the United States within twenty-four hours of its release.

The original seven books were adapted into an eight-part namesake film series by Warner Bros. Pictures. In 2016, the total value of the Harry Potter franchise was estimated at $25 billion,[7] making Harry Potter one of the highest-grossing media franchises of all time. Harry Potter and the Cursed Child is a play based on a story co-written by Rowling.

The success of the books and films has allowed the Harry Potter franchise to expand with numerous derivative works, a travelling exhibition that premiered in Chicago in 2009, a studio tour in London that opened in 2012, a digital platform on which J. K. Rowling updates the series with new information and insight, and a pentalogy of spin-off films premiering in November 2016 with Fantastic Beasts and Where to Find Them, among many other developments. Themed attractions, collectively known as The Wizarding World of Harry Potter, have been built at several Universal Parks & Resorts amusement parks around the world."""

In [33]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    lemma = WordNetLemmatizer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = lemma.lemmatize(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [34]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [35]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [36]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [37]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [38]:
def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

In [39]:
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

In [40]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [42]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mihir\AppData\Roaming\nltk_data...


True

In [43]:
import math

from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords    
    
'''
We already have a sentence tokenizer, so we just need 
to run the sent_tokenize() method to create the array of sentences.
'''
# 1 Sentence Tokenize
sentences = sent_tokenize(text)
total_documents = len(sentences)
#print(sentences)

# 2 Create the Frequency matrix of the words in each sentence.
freq_matrix = _create_frequency_matrix(sentences)
#print(freq_matrix)

'''
Term frequency (TF) is how often a word appears in a document, divided by how many words are there in a document.
'''
# 3 Calculate TermFrequency and generate a matrix
tf_matrix = _create_tf_matrix(freq_matrix)
#print(tf_matrix)

# 4 creating table for documents per words
count_doc_per_words = _create_documents_per_words(freq_matrix)
#print(count_doc_per_words)

'''
Inverse document frequency (IDF) is how unique or rare a word is.
'''
# 5 Calculate IDF and generate a matrix
idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
#print(idf_matrix)

# 6 Calculate TF-IDF and generate a matrix
tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
#print(tf_idf_matrix)

# 7 Important Algorithm: score the sentences
sentence_scores = _score_sentences(tf_idf_matrix)
#print(sentence_scores)

# 8 Find the threshold
threshold = _find_average_score(sentence_scores)
#print(threshold)

# 9 Important Algorithm: Generate the summary
summary = _generate_summary(sentences, sentence_scores, threshold)
print(summary)

 The series was originally published in English by Bloomsbury in the United Kingdom and Scholastic Press in the United States. All versions around the world are printed by Grafica Veneta in Italy. [2] According to Rowling, the main theme is death. [3] Other major themes in the series include prejudice, corruption, and madness. They have attracted a wide adult audience as well as younger readers and are often considered cornerstones of modern young adult literature. The original seven books were adapted into an eight-part namesake film series by Warner Bros. Pictures. Harry Potter and the Cursed Child is a play based on a story co-written by Rowling.


In [48]:
from collections import Counter
dc = {}
for i in tf_idf_matrix:
    for j in tf_idf_matrix[i]:
        dc[j] = tf_idf_matrix[i][j]
fin_max = max(dc, key=dc.get)
print("Maximum value:",fin_max)
print(Counter(dc).most_common(5))

Maximum value: adult
[('adult', 0.1757784173397534), ('version', 0.15380611517228424), ('printed', 0.15380611517228424), ('grafica', 0.15380611517228424), ('veneta', 0.15380611517228424)]


In [2]:
!pip install summa

  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54411 sha256=f430a7409905bd23344561117acb8d95636ba7e16289a1543990bcd58af87b3b
  Stored in directory: c:\users\mihir\appdata\local\pip\cache\wheels\ed\2c\5f\a0ccc5955d44d2cea78729f4425e73f818d2629517f7af0f8b
Successfully built summa


In [51]:
from summa import keywords

In [52]:
TR_keywords = keywords.keywords(summary, scores=True)
print(TR_keywords[0:10])

[('adult', 0.3380449104576557), ('theme', 0.21790166957939905), ('themes', 0.21790166957939905), ('united', 0.21790166957939883), ('originally', 0.19383195948940687), ('original seven', 0.19383195948940676), ('bros', 0.1836023327011601), ('harry', 0.18360233270115994), ('film', 0.17862367456062173)]


In [6]:
!pip install keybert

  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  Created wheel for keybert: filename=keybert-0.5.1-py3-none-any.whl size=21332 sha256=7f4da0a0827a80cbf122ed41c197f6d28d1deb5a869f7f4cff100b95bf72ddd3
  Stored in directory: c:\users\mihir\appdata\local\pip\cache\wheels\94\18\2a\f26bbcd25924aab452bb4bcc2345a55c07160823d196a264c7
Successfully built keybert


In [7]:
from keybert import KeyBERT

In [8]:
kw_model = KeyBERT(model='all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [49]:
keywords = kw_model.extract_keywords(summary, 

                                     keyphrase_ngram_range=(1, 3), 

                                     stop_words='english', 

                                     highlight=False,

                                     top_n=10)

keywords_list= list(dict(keywords).keys())

print(keywords_list)

['potter cursed child', 'harry potter cursed', 'rowling main theme', 'potter cursed', 'story written rowling', 'cursed child play', 'harry potter', 'cursed child', 'original seven books', 'written rowling']
